In [1]:
import numpy as np
import pandas as pd
import os
from scipy import stats
import utils
import torch.optim as optim
import torch
from torch.utils.data.sampler import RandomSampler

import model.net as net
from dataloader import *
from train import train_and_evaluate

months = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

In [6]:
data_set = 'London_2013'
path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
path = path.replace('\\', '/')

data = get_data(path, data_set)

100%|██████████████████████████████████████████████████████████████████████████████| 2002/2002 [02:24<00:00, 13.88it/s]


In [13]:
month = 1
n_clusters = 2
method = 'hierarchical/euclidean'

path_cluster = os.path.join(path, 'result', data_set, 'clustering', 'point', method, f'n_clusters_{n_clusters}.csv')
clusters = pd.read_csv(path_cluster, header=None)
path_data = os.path.join(path, 'data', 'deepar')

series = data[:, month-1, :months[month-1]*24]

weather = get_weather(path, data_set, month)
week = get_dow(data_set, month)
day = get_hod(month)

num_covariates = 4
covariates = np.zeros((num_covariates, len(series[0])))
covariates[1] = stats.zscore(weather)
covariates[2] = stats.zscore(week)
covariates[3] = stats.zscore(day)
covariates = covariates.T

In [ ]:
for i in range(n_clusters):

    index = list(clusters[month-1] == i)
    sub_series = series[index]
    
    test_data = sub_series[:, -168*2:].T
    train_data = sub_series[:, :-168].T
    
    data_start = (train_data != 0).argmax(axis=0)
    total_time = sub_series.shape[1]
    num_series = sub_series.shape[0]
    
    window_size = 192
    stride_size = 24
    
    # prepare data
    cov = covariates.copy()
    train_x_input, train_v_input, train_label = prep_data(train_data, cov, data_start, window_size, stride_size, num_covariates, num_series, total_time)
    cov = covariates.copy()
    test_x_input, test_v_input, test_label = prep_data(test_data, cov, data_start, window_size, stride_size, num_covariates, num_series, total_time, train=False)
    
    # params
    json_path = os.path.join(path, 'forecasting', 'deepar', 'params.json')
    params = utils.Params(json_path)
    
    params.num_class = np.sum(index)
    params.relative_metrics = False
    params.sampling = False
    
    # use GPU if available
    cuda_exist = torch.cuda.is_available()
    # Set random seeds for reproducible experiments if necessary
    if cuda_exist:
        params.device = torch.device('cuda')
        # torch.cuda.manual_seed(240)
        model = net.Net(params).cuda()
    else:
        params.device = torch.device('cpu')
        # torch.manual_seed(230)
        model = net.Net(params)
    
    train_set = TrainDataset(train_x_input, train_label)
    test_set = TestDataset(test_x_input, test_v_input, test_label)
    sampler = WeightedSampler(train_v_input) # Use weighted sampler instead of random sampler
    train_loader = DataLoader(train_set, batch_size=params.batch_size, sampler=sampler, num_workers=4)
    test_loader = DataLoader(test_set, batch_size=params.predict_batch, sampler=RandomSampler(test_set), num_workers=4)
    
    optimizer = optim.Adam(model.parameters(), lr=params.learning_rate)
    loss_fn = net.loss_fn
    
    restore_file = None
    train_and_evaluate(model,
                       train_loader,
                       test_loader,
                       optimizer,
                       loss_fn,
                       params,
                       restore_file)
    break

begin training and evaluation
Epoch 1/20
train_loss: 0.810928742090861
train_loss: 0.39846479892730713
train_loss: 0.26568132638931274
train_loss: 0.25202178955078125
train_loss: 0.1933418313662211
train_loss: 0.27647175391515094
train_loss: 0.14044452706972757
train_loss: 0.12797550360361734
train_loss: 0.18297197421391806
train_loss: 0.09055677056312561
train_loss: 0.203786830107371
train_loss: 0.04773290455341339
train_loss: 0.0803909848133723
train_loss: 0.0755847046772639
train_loss: -0.01631621519724528
train_loss: -0.05472126603126526
train_loss: 0.0816812664270401
train_loss: -0.003837788477540016
train_loss: -0.09110777576764424
train_loss: 0.008345970263083776
train_loss: -0.05964496235052744
train_loss: -0.12200747927029927
train_loss: -0.11944946646690369
train_loss: -0.1175386905670166
train_loss: -0.0817738672097524
train_loss: -0.07605311771233876
train_loss: -0.20074248313903809
train_loss: -0.18345165252685547
train_loss: -0.1818032662073771
train_loss: -0.166408260663